<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/test_with_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorboard
from datetime import datetime
%load_ext tensorboard

# Tensorboard for visualising computational graph

In [ ]:
!rm -rf ./logs/
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)
%tensorboard --logdir logs/func

## Tensorboard example

In [ ]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

# Test with RNN
Try to understand how to use batch and how backprop works under the hood

In [1]:
from numpy.random import seed
seed(1)
tf.random.set_seed(1)
rnn0 = tf.keras.layers.LSTM(units = 3, return_sequences=True)

rnn = tf.keras.layers.LSTM(units = 1)
input = tf.constant([
                     [[1, -1]], 
                     [[0, -1]],
                     [[0, -1]],
                     [[0, -1]],
                    ], dtype=tf.float32)
neig = tf.constant([
                     [0, 1, 0, 0], 
                     [1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                    ], dtype=tf.float32)

feature = tf.constant([
                     [1], 
                     [0],
                     [0],
                     [0],
                    ], dtype=tf.float32)

ground = tf.constant([
                     [1, 0], 
                     [0, 1],
                     [0, 2],
                     [0, 3],
                    ], dtype=tf.float32)

@tf.function
def my_func(x, rnn):
  # A simple hand-rolled layer.
  return rnn(rnn0(x))

tf.summary.trace_on(graph=True, profiler=True)
loss = tf.losses.mse
optimizer = tf.optimizers.SGD()

for j in range(10000):
  with tf.GradientTape() as tape:

    result = input
    for i in range(1):
      reshape = tf.reshape(input[:,:, 1], [4, 1])
      neigh_evaluation = tf.reduce_min(tf.multiply(neig, reshape), 1)
      input_network = tf.concat([input[:,:, 1], neigh_evaluation[:, tf.newaxis]], 1)
      result = input_network[:, tf.newaxis]
      result = my_func(result, rnn)
      other = tf.concat([feature, result], axis = 1)
      result = tf.expand_dims(other, [1])
    to_backprop = loss(ground, result[:, 0, :])
    rnn_g, rnn0_g = tape.gradient(-result, [rnn.weights, rnn0.weights])
    optimizer.apply_gradients(zip(rnn_g, rnn.weights))
    optimizer.apply_gradients(zip(rnn0_g, rnn0.weights))
    print(tf.reduce_sum(to_backprop))

with writer.as_default():
  tf.summary.trace_export(
      name="rnn",
      step=0,
      profiler_outdir=logdir)

NameError: ignored